##  Covid-19 in Seoul
* http://www.seoul.go.kr/coronaV/coronaStatus.do

In [186]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from fake_useragent import UserAgent
import re
import time
from selenium.webdriver.firefox.options import Options

In [202]:
target_url = 'http://www.seoul.go.kr/coronaV/coronaStatus.do'
browser_local_url = r'C:/Program Files/Mozilla Firefox/firefox.exe'
selenium_driver_local_url = r'./driver/geckodriver.exe'

In [203]:
capabilities = webdriver.DesiredCapabilities().FIREFOX
capabilities["marionette"] = True
binary = FirefoxBinary(browser_local_url)
driver = webdriver.Firefox(firefox_binary = binary, 
                           capabilities = capabilities, 
                           executable_path = selenium_driver_local_url)

In [204]:
driver.get(target_url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [205]:
page_nums = soup.findAll("a", {'class':'paginate_button'})

In [206]:
page_num_list = []
for i in page_nums:
    try:
        temp_page = int(i.get_text())
        if not temp_page in page_num_list:
            page_num_list.append(temp_page)
    except:
        continue
end_of_page_num = max(page_num_list)
print("End of the page num: %d" %(end_of_page_num))

End of the page num: 17


In [207]:
df = pd.DataFrame(columns = ['seoul_patient_num', 'national_patient_num', 'day_of_detection', 'address', 'travel', 'exposure', 'treatment'])
df

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment


In [208]:
row_index = 0
for page_num in range(4, end_of_page_num+1):
    if page_num <= 5:
        print("Crawling at webpage %d" %(page_num))
        driver.get(target_url)
        driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(page_num)  +']').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("Crawling at webpage %d" %(page_num))
        driver.get(target_url)
        search_start_page = 4
        while True:
            try:
                print("here",page_num)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')   
                crr_page_num = soup.findAll("a", {'class':'paginate_button'}).findAll('a', {'data-dt-idx' : "3"})
                
                break
            except:
                print("there", search_start_page)
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, 
                                                                                 '//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(search_start_page)  +']'))).click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
    
    parsed_soup = soup.findAll('table')[6]
    for i in parsed_soup.findAll('tr'):
        col_cnt = 0
        if len(i.findAll(['p', 'td'])) == 0:
            continue
        for col_cnt, j in enumerate(i.findAll(['p', 'td'])):
            if col_cnt == 0:
                df.loc[row_index, 'seoul_patient_num'] = j.get_text()
            elif col_cnt == 1:
                df.loc[row_index, 'national_patient_num'] = j.get_text()
            elif col_cnt == 2:
                df.loc[row_index, 'day_of_detection'] = j.get_text()
            elif col_cnt == 3:
                df.loc[row_index, 'address'] = j.get_text()
            elif col_cnt == 4:
                df.loc[row_index, 'travel'] = j.get_text()
            elif col_cnt == 5:
                df.loc[row_index, 'exposure'] = j.get_text()
            else:
                df.loc[row_index, 'treatment'] = j.get_text()
        row_index = row_index+1  

Crawling at webpage 4
Crawling at webpage 5
Crawling at webpage 6
here 6
Crawling at webpage 7
here 7
there 5
here 7
there 6


TimeoutException: Message: 


In [153]:
df

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment
0,1612,14388,8.2.,동대문구,-,타시도 확진자 접촉,서울의료원
1,1611,14386,8.2.,대구,이라크,해외 접촉 추정,국립중앙의료원
2,1610,14383,8.2.,강서구,필리핀,해외 접촉 추정,서북병원
3,1609,14381,8.2.,성북구,-,확인 중,보라매병원
4,1608,14382,8.2.,성남시,일본,해외 접촉 추정,서울백병원
...,...,...,...,...,...,...,...
195,1417,13439,7.12.,관악구,-,강남구 사무실 관련,보라매병원(퇴원)
196,1416,13412,7.11.,강동구,-,강남구 이동통신 종사자,서울의료원(퇴원)
197,1415,13396,7.11.,강동구,-,강남구 이동통신 종사자,한양대학교병원(퇴원)
198,1414,13398,7.11.,중랑구,-,확인 중,서울의료원(퇴원)


In [8]:
url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
url

'http://www.seoul.go.kr/coronaV/coronaStatus.do'

In [9]:
table = pd.read_html(url)
len(table)

8

In [10]:
table[0].T

,0,1,2
종로구,29,마포구,53
중구,20,양천구,72
용산구,54,강서구,106
성동구,57,구로구,91
광진구,22,금천구,39
동대문구,45,영등포구,72
중랑구,44,동작구,70
성북구,43,관악구,144
강북구,28,서초구,64
도봉구,64,강남구,98


In [11]:
df = table[4]
df.shape

(1, 1)

In [12]:
df.head()

,0
0,< 신종 코로나바이러스 감염증 예방 의료기관 행동수칙 >○ 호흡기 질환자 진료시 마...


In [13]:
last_day = df.loc[0, "확진일"]
last_day = last_day.replace(".", "_")
last_day

KeyError: '확진일'

In [8]:
file_name = f"seoul_covid_{last_day}.csv"
file_name

'seoul_covid_6_27_.csv'

In [9]:
df.to_csv(file_name, index=False)

In [10]:
pd.read_csv(file_name)

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1298,12696,6.27.,송파구,-,강남구 역삼동 모임,서울대학병원
1,1297,12693,6.27.,영등포구,-,확인 중,서남병원
2,1296,12694,6.27.,관악구,-,확인 중,강남성심병원
3,1295,12690,6.27.,관악구,-,왕성교회 관련,서남병원
4,1294,12688,6.27.,관악구,-,왕성교회 관련,서남병원
...,...,...,...,...,...,...,...
1293,5,9,1.31.,성북구,-,#5 접촉,서울의료원(퇴원)
1294,4,7,1.30.,마포구,중국 우한시,해외 접촉,서울의료원(퇴원)
1295,3,6,1.30.,종로구,-,#3 접촉,서울대학교병원(퇴원)
1296,2,5,1.30.,중랑구,중국 우한시,해외 접촉,서울의료원(퇴원)


In [11]:
df = df.sort_values(["연번"], ascending=False)

In [12]:
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1298,12696,6.27.,송파구,-,강남구 역삼동 모임,서울대학병원
1,1297,12693,6.27.,영등포구,-,확인 중,서남병원
2,1296,12694,6.27.,관악구,-,확인 중,강남성심병원
3,1295,12690,6.27.,관악구,-,왕성교회 관련,서남병원
4,1294,12688,6.27.,관악구,-,왕성교회 관련,서남병원


In [13]:
df.tail()

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
1293,5,9,1.31.,성북구,-,#5 접촉,서울의료원(퇴원)
1294,4,7,1.30.,마포구,중국 우한시,해외 접촉,서울의료원(퇴원)
1295,3,6,1.30.,종로구,-,#3 접촉,서울대학교병원(퇴원)
1296,2,5,1.30.,중랑구,중국 우한시,해외 접촉,서울의료원(퇴원)
1297,1,2,1.24.,강서구,중국 우한시,해외 접촉,국립중앙의료원(퇴원)
